In [ ]:
# Téléchargement des données : Kaggle mot de passe et login

import urllib.request as ur
#ur.urlretrieve ("https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/downloads/WA_Fn-UseC_-HR-Employee-Attrition.csv"
#                , "WA_Fn-UseC_-HR-Employee-Attrition.zip")

In [8]:
import pandas as pd
import numpy as np

df = pd.read_table("WA_Fn-UseC_-HR-Employee-Attrition.csv", sep = ",")

print('Columns', df.columns)
print(df.head())
print(df.isnull().sum())

Columns Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')
   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & 

# First observations :

_On the structure:_
- We have 35 different variables : preprocessing needed to get rid of the useless variables (the ones with no correlation with Attrition)
- A lot of qalitative variables, we might lose a lot of information with PCA
- No missing values in this dataset

_On the datas themselves:_
- What do they represent?
    - DailyRate
    - EmployeeCount
    - HourlyRate
    - Over18
    - PercentSalaryHike (unit?)
    


In [10]:
# What is the unit of DailyRate ?
print(df["DailyRate"].describe())
# 802 = 8:02 ? en moyenne, fait sens
#mais min et max impossibles (1h/jour vs 14h/jour ?)

print(df["HourlyRate"].describe())

# HourlyRate en % : représente le taux d'efficacité par heure ?

count    1470.000000
mean      802.485714
std       403.509100
min       102.000000
25%       465.000000
50%       802.000000
75%      1157.000000
max      1499.000000
Name: DailyRate, dtype: float64
count    1470.000000
mean       65.891156
std        20.329428
min        30.000000
25%        48.000000
50%        66.000000
75%        83.750000
max       100.000000
Name: HourlyRate, dtype: float64
